#Imports

In [1]:
#Import das libs
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

# Display dos parâmetros
from IPython.display import display
from ipywidgets import interact, IntSlider

# Separação dos dados
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import TimeSeriesSplit 

# Modelagem
from sklearn.metrics import (explained_variance_score, 
                             mean_absolute_error, 
                             mean_squared_error, 
                             mean_squared_log_error,
                             r2_score)
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV  

In [2]:
# Carga dos dados 
df_hist = pd.read_csv('./Dados/Histórico_Demanda_Elétrica_Peru_2006-2021.csv', parse_dates=['date'])
df_tempo = pd.read_csv('./Dados/Tempo_Lima_2006-2021.csv',delimiter=',')
print(df_hist.shape)
print(df_tempo.shape)

(537696, 2)
(116934, 7)


#1 - Análise Exploratória


Para realizar essa etapa deve-se ter em mente as duas perguntas:
<ol>
<li> Como se comporta a demanda elétrica ao longo dos anos, meses, semanas, dias...?</li>
<li> Como se dá a ocorrência do pico intra diário?</li>

Vamos analisar cada um dos datasets separadamente, inicialmente.

## 1.1 - Histórico

**Histórico da demanda**: série temporal com o histórico da demanda de potência
elétrica da rede peruana com intervalo de amostragem de 15 minutos. A seguir
um descritivo breve das colunas presentes na base de dados.
<ul>
  <li>a. Nome do arquivo: Histórico Potência.csv</li>
  <li>b. Colunas:</li>
    <ul>
      <li>i. FECHA: momento da amostragem da demanda energética</li>
      <li>ii. EJECUTADO: valor da demanda de potência da rede elétrica peruana em MW </li>
    </ul>
</ul>

In [3]:
df_hist.head()

,date,executado
0,2006-01-01 00:00:00,2069.01876
1,2006-01-01 00:15:00,2733.44840
2,2006-01-01 00:30:00,2681.27092
3,2006-01-01 00:45:00,2633.54249
4,2006-01-01 01:00:00,2611.28745


In [4]:
df_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537696 entries, 0 to 537695
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date       537696 non-null  datetime64[ns]
 1   executado  537696 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 8.2 MB


## 1.2 - Tempo

**Dados meteorológicos**: histórico de temperatura e umidade da cidade mais
populosa do Peru (Lima), com intervalo de amostragem de 1 hora. A seguir um
descritivo breve das colunas presentes na base de dados.
<ul>
  <li>a. Nome do arquivo: Temperatura Lima 2018 - 2021.csv</li>
  <li>b. Colunas</li>
    <ul>
      <li>i. FECHA: momento da amostragem dos dados meteorológicos</li>
      <li>ii. TEMPERATURA (°C): temperatura medida, em graus Celsius (°C)</li>
      <li>iii. PRECIPITACIÓN (mm/hora): precipitação medida, em milímetro por hora (mm/hora)</li>
      <li> iv. HUMEDAD (%): umidade medida, em porcentagem (%)</li>
      <li> v. DIRECCION DEL VIENTO (°): direção do vento, em graus (o)</li>
      <li>vi. VELOCIDAD DEL VIENTO (m/s): velocidade do vento, m/s</li>

O primeiro problema problema que foi encontrado foi na instância 61564 que apresenta um erro em seus valores que será tratado de forma a apenas colocar seus valores de 'DATE' e 'HOUR', anulando o restante.

In [5]:
list_61564 = df_tempo.loc[61564,:]['DATE'].split(',')
df_tempo.loc[61564,'DATE'] = list_61564[0]
df_tempo.loc[61564,'HOUR'] = list_61564[1]

Ao começar a lidar com o dataset de Tempo, percebeu-se alguns erros tratados a seguir, primeiramente, percebe-se que há instâncias não apresentam valor na coluna hora e que esse valor está junto à coluna date, a seguir isso é corrigido para que por todo o dataset tenhamos as colunas de hora e data separadamente.

In [6]:
df_tempo['DATE'] = pd.to_datetime(df_tempo['DATE'], errors='raise')
df_tempo['HOUR'] = pd.to_datetime(df_tempo['HOUR'], errors='raise')

In [7]:
df_tempo['HOUR'] = df_tempo['HOUR'].dt.time

In [8]:
subst = df_tempo[df_tempo['HOUR'].isnull() == True]['DATE'].dt.time
lista_nulos = list(df_tempo[df_tempo['HOUR'].isnull() == True]['HOUR'].index)
df_tempo.loc[lista_nulos, 'HOUR'] = subst

Uma primeira observação é que, com exceção das colunas de data e hora, o restante das colunas apresentam um número significativo de valores nulos.

In [18]:
df_tempo.head()

,DATE,HOUR,TEMP,PREC_H,HUM,W_DIR,W_VEL
0,2006-01-01,00:00:00,NaN,NaN,NaN,NaN,NaN
1,2006-01-02,00:00:00,NaN,NaN,NaN,NaN,NaN
2,2006-01-03,00:00:00,NaN,NaN,NaN,NaN,NaN
3,2006-01-04,00:00:00,NaN,NaN,NaN,NaN,NaN
4,2006-01-05,00:00:00,NaN,NaN,NaN,NaN,NaN


Uma importante observação é que a coluna 'HOUR' foi convertida para um tipo 'datetime.time' que aparece aqui como um 'dtype' de 'object'.

In [19]:
df_tempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116934 entries, 0 to 116933
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   DATE    116934 non-null  datetime64[ns]
 1   HOUR    116934 non-null  object        
 2   TEMP    98744 non-null   float64       
 3   PREC_H  72417 non-null   float64       
 4   HUM     98773 non-null   float64       
 5   W_DIR   98758 non-null   float64       
 6   W_VEL   98760 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 6.2+ MB


#2 - Feature engineering

In [10]:
### Perguntas Direcionadoras ###
#Quais são as features que podem ajudar a prever a demanda elétrica?
#Lembrem-se de correlação vs causalidade

#3 - Construção do Modelo

In [11]:
#Dica 1: pense nas métricas de sucesso
#Dica 2: pense numa janela para o test set coerente com o problema (sugestão: utilize pelo menos 1 mês)
#Dica 3: comece com modelos mais simples
#ATENÇÃO!: cuidado com vazamento de dados do train set para o test set!

#4 - Backtest

In [12]:
#Dica 1: pense numa janela para o test set coerente com o problema (sugestão: utilize pelo menos 1 mês)
#Dica 2: construa uma função para automatizar esse trabalho
#ATENÇÃO!: cuidado com vazamento de dados do train set para o test set!

#5 - Previsão do dia  03/05/2021

In [13]:
#Lembre-se de salvar a previsão com o nome do arquivo seguindo o padrão 'VAI_previsao_M6_nome_sobrenome.csv'
#e com a estrutura do arquivo seguindo o mesmo padrão do data set 'Histórico Demanda Elétrica Peru 2006 - 2021.csv'